### generate mock point source data
#### main function ：F(x)
- input x : 
  - [Fe/H], Age, (DM), $\rm{f}_{b}$, IMF($\alpha$)
  - m(Error), mag range, $\rm{N}_{partical}$

- output y :
  - G, BP-RP ==> CMD

#### functions inside F(x) :
1. isochrone([Fe/H], Age, (DM))
  
2. Sample mockstar(IMF, $\rm{f}_{b}$)
 
3. add Error 

In [50]:
from berliner import CMD
from scipy import integrate
import random
import pandas as pd
import numpy as np
import emcee

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("default")

import sys
sys.path.append('../') # 添加包含functionkit.py文件的目录路径
import functionkit

#### 1. isochrone([Fe/H], Age, (DM))

- get isochrones using berliner 

In [52]:
# get isochrones using berliner
c = CMD()
# define your grid of logAge and [M/H] as tuple(lower, upper, step)
grid_logage = (6, 10.2, 0.1)
grid_mh = (-2.6, 0.5, 0.1)
# download isochrones in parallel
isoc_lgage, isoc_mhini, isoc_list = c.get_isochrone_grid_mh(
    grid_logage=grid_logage, grid_mh=grid_mh, photsys_file="gaiaDR2",
    n_jobs=50, verbose=10)

@CMD: n(logAge)=43, n([M/H])=32
@CMD: grid(logAge):  [ 6.   6.1  6.2  6.3  6.4  6.5  6.6  6.7  6.8  6.9  7.   7.1  7.2  7.3
  7.4  7.5  7.6  7.7  7.8  7.9  8.   8.1  8.2  8.3  8.4  8.5  8.6  8.7
  8.8  8.9  9.   9.1  9.2  9.3  9.4  9.5  9.6  9.7  9.8  9.9 10.  10.1
 10.2]
@CMD: grid([M/H]) :  [-2.60000000e+00 -2.50000000e+00 -2.40000000e+00 -2.30000000e+00
 -2.20000000e+00 -2.10000000e+00 -2.00000000e+00 -1.90000000e+00
 -1.80000000e+00 -1.70000000e+00 -1.60000000e+00 -1.50000000e+00
 -1.40000000e+00 -1.30000000e+00 -1.20000000e+00 -1.10000000e+00
 -1.00000000e+00 -9.00000000e-01 -8.00000000e-01 -7.00000000e-01
 -6.00000000e-01 -5.00000000e-01 -4.00000000e-01 -3.00000000e-01
 -2.00000000e-01 -1.00000000e-01  2.22044605e-15  1.00000000e-01
  2.00000000e-01  3.00000000e-01  4.00000000e-01  5.00000000e-01]


[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done   4 out of  43 | elapsed:  1.5min remaining: 14.4min
[Parallel(n_jobs=50)]: Done   9 out of  43 | elapsed:  1.6min remaining:  6.2min
[Parallel(n_jobs=50)]: Done  14 out of  43 | elapsed:  1.8min remaining:  3.7min
[Parallel(n_jobs=50)]: Done  19 out of  43 | elapsed:  1.8min remaining:  2.3min
[Parallel(n_jobs=50)]: Done  24 out of  43 | elapsed:  1.8min remaining:  1.4min
[Parallel(n_jobs=50)]: Done  29 out of  43 | elapsed:  1.8min remaining:   53.5s
[Parallel(n_jobs=50)]: Done  34 out of  43 | elapsed:  1.9min remaining:   29.5s
[Parallel(n_jobs=50)]: Done  39 out of  43 | elapsed:  1.9min remaining:   11.6s
[Parallel(n_jobs=50)]: Done  43 out of  43 | elapsed:  1.9min finished


In [53]:
for i in range(len(isoc_list)):
    iso = isoc_list[i].to_pandas()
    #iso = iso[(iso['label']>0)&(iso['label']<=7)]
    logAge = iso.iloc[0]['logAge']
    MH = iso.iloc[0]['MH']
    iso.to_csv('../../data/isochrones/iso_age_%s_mh_%s.csv'%(logAge,MH),index=False)

#### 2. Sample mockstar(IMF, $\rm{f}_{b}$)

- define different IMF `fun_IMF(m_x,label)` and `pdf_IMF(m_x,m_min,m_max,label)`

- ramdon sampling according to defined IMF `random_IMF(m_n, m_min, m_max,label)`

In [ ]:
# 初始质量函数
# generate the mass catalog follow the selected IMF 
def fun_IMF(m_x,label='Kroupa'):
    '''
    Initial Mass Funcation
    label = ['Salpeter','Kroupa','Chabrier']
    '''
    while label == 'Salpeter':
        if m_x < 0.1:
            return 0
        elif m_x < 100:
            return m_x**(-2.35)
        else : 
            return 0
    
    while label == 'Kroupa':
        if m_x < 0.08 : 
            return 0
        elif m_x < 0.5 : 
            return 2*m_x**(-1.3)
        elif m_x < 150:
            return m_x**(-2.3)
        else :
            return 0
        
        
    while label == 'Chabrier':
        if m_x < 0.07 : 
            return 0
        elif m_x < 1.0 : 
            return m_x**(-1.55)
        elif m_x < 100 : 
            return m_x**(-2.7)
        else : 
            return 0

# 返回IMF的pdf       
def pdf_IMF(m_x,m_min,m_max,label='Kroupa'):
    if m_x < m_min or m_x > m_max:
        return 0
    else : 
        return fun_IMF(m_x,label)/integrate.quad(lambda x : fun_IMF(x,label), m_min, m_max)[0]

# 按照 初始质量函数 随机生成 m_n 个范围在 (m_min,m_max) 之间的质量 
def random_IMF(m_n, m_min, m_max,label='Kroupa'):
    m_result = []
    c = pdf_IMF(m_min, m_min, m_max, label)
    for i in range(m_n):
        m_flag = 0 # 标志，判断生成的 mass 是否已经到达设定的质量上限 m_max
        while m_flag == 0:
            m_x = random.uniform(m_min, m_max) # 从(m_min, m_max)的均匀分布中随机抽取一个数
            m_y = random.uniform(0,1) 
            if m_y < pdf_IMF(m_x, m_min, m_max, label)/c: 
                m_result.append(m_x)
                m_flag = 1
    return m_result